In [265]:
'''

Analysis of Student Loan Refinance Product

In this analysis I am going to explore the student loan refinance product; specifically, I am going to try to tease
out analysis of different age, marketing and credit score cohorts.

Process Overview: https://docs.google.com/document/d/1lOzoz94xiYFPokeLDsXEc7A2QNE3bO4wviIUCshwOlo/edit?usp=sharing
Analysis Prompt: https://docs.google.com/document/d/1nDPkgkobae51odJwx_Yn3UIzNFLJkfQKfgHy3Rke6qI/edit?usp=sharing

'''
%matplotlib inline
import pandas as pd
from plotly import graph_objects as go

In [217]:
'''
Read in data from csv
'''
df = pd.read_csv('Product_Analyst_Data - Data.csv')

In [264]:
'''

Initial Dataframe

'''
df.head()

,submission_id,marketing_channel,loan_amount,age,income,fico,prequal_completed,prequal_rates_received,cosigner_invited,cosigner_active,full_form_in_progress,full_form_completed,offer_accepted,age_group,fico_group,loan_amount_group
0,2644866,Organic,26657.0,25,39520.0,747,1,1,0,0,1,0,0,25 to 35,720 to 850,25000 to 45000
1,1946013,Partner,57947.0,34,45000.0,784,1,1,0,0,0,0,0,25 to 35,720 to 850,45000 to 80000
2,2348051,Partner,36172.0,32,90000.0,697,1,1,1,0,1,0,0,25 to 35,690 to 719,25000 to 45000
3,2959388,Organic,35000.0,33,31200.0,708,1,1,0,0,0,0,0,25 to 35,690 to 719,25000 to 45000
4,1962239,Organic,35000.0,36,85000.0,771,1,1,0,0,0,0,0,35 to 55,720 to 850,25000 to 45000


In [219]:
'''

Summary statistics to understand the numeric data

'''
df[['loan_amount','age','income','fico']].describe()

,loan_amount,age,income,fico
count,5000.000000,5000.000000,5000.000000,5000.000000
mean,61767.671674,32.966600,71341.078608,721.555200
std,56532.941878,8.730815,52456.128480,73.229004
min,60.000000,18.000000,1.000000,449.000000
25%,24806.000000,27.000000,40000.000000,681.000000
50%,45000.000000,31.000000,60000.000000,732.000000
75%,80000.000000,36.000000,85000.000000,776.000000
max,467720.000000,80.000000,750000.000000,850.000000


In [220]:
'''

Summary statistics by marketing channel

'''

df.groupby('marketing_channel')['loan_amount','age','income','fico'].mean()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,loan_amount,age,income,fico
marketing_channel,,,,
Organic,63467.560356,31.859907,70037.839009,722.867647
Paid,57054.286665,33.098235,67978.102659,713.947059
Partner,64664.334193,33.567231,75026.758227,727.151892


In [221]:
'''

The following functions are applied on the rows to assign age, fico and loan amount groups to rows. 

'''

def age_group(row):
    #The age groupings were created to fall into four general age groups based on stage of life e.g. student, professional
    if row['age'] > 18 and row['age'] < 25:
        return '18 to 25'
    elif row['age'] >= 25 and row['age'] < 35:
        return '25 to 35' 
    elif row['age'] >= 35 and row['age'] < 55:
        return '35 to 55' 
    else:
        return 'over 55'

def fico_group(row):
    #The fico groups are based on the following credit score ranges https://www.cafecredit.com/credit-score-range
    if row['fico'] >= 720:
        return '720 to 850'
    elif row['fico'] >= 690 and row['fico'] < 720:
        return '690 to 719' 
    elif row['fico'] >= 630 and row['age'] < 690:
        return '630 to 689' 
    else:
        return 'less than 630'

    

def loan_amount_group(row):
    #The loan amounts are based roughly on the distribution 25%, 50% and 75% quartiles
    if row['loan_amount'] >= 80000:
        return 'over 80000'
    elif row['loan_amount'] >= 45000 and row['loan_amount'] < 80000:
        return '45000 to 80000' 
    elif row['loan_amount'] >= 25000 and row['loan_amount'] < 45000:
        return '25000 to 45000' 
    else:
        return 'less than 25000'
df['age_group'] = df.apply(age_group, axis=1)
df['fico_group'] = df.apply(fico_group, axis=1)
df['loan_amount_group'] = df.apply(loan_amount_group,axis=1)
df.head()

,submission_id,marketing_channel,loan_amount,age,income,fico,prequal_completed,prequal_rates_received,cosigner_invited,cosigner_active,full_form_in_progress,full_form_completed,offer_accepted,age_group,fico_group,loan_amount_group
0,2644866,Organic,26657.0,25,39520.0,747,1,1,0,0,1,0,0,25 to 35,720 to 850,25000 to 45000
1,1946013,Partner,57947.0,34,45000.0,784,1,1,0,0,0,0,0,25 to 35,720 to 850,45000 to 80000
2,2348051,Partner,36172.0,32,90000.0,697,1,1,1,0,1,0,0,25 to 35,690 to 719,25000 to 45000
3,2959388,Organic,35000.0,33,31200.0,708,1,1,0,0,0,0,0,25 to 35,690 to 719,25000 to 45000
4,1962239,Organic,35000.0,36,85000.0,771,1,1,0,0,0,0,0,35 to 55,720 to 850,25000 to 45000


In [222]:
'''

Group the funnel data by marketing channel. 

'''


marketing_df = df.groupby('marketing_channel')['prequal_completed','prequal_rates_received','cosigner_invited','cosigner_active','full_form_in_progress','full_form_completed','offer_accepted'].sum()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [223]:
'''

Removed co-signers for ease of viewing. Will review co-signer effects later in analysis

'''

marketing_df = marketing_df.drop(['cosigner_invited','cosigner_active'], axis=1)

In [224]:
'''

Setup variables for plot

'''


organic = marketing_df[:1].values
paid = marketing_df[1:2].values
partner = marketing_df[2:3].values

In [273]:
'''

Plot Setup: Marketing

'''


from plotly import graph_objects as go

fig = go.Figure()

fig.add_trace(go.Funnel(
    name = 'Organic',
    y = ['prequal_completed','prequal_rates_received','full_form_in_progress','full_form_completed','offer_accepted'],
    x = organic[0],
    textinfo = "value+percent initial"))

fig.add_trace(go.Funnel(
    name = 'Paid',
    orientation = "h",
    y = ['prequal_completed','prequal_rates_received','full_form_in_progress','full_form_completed','offer_accepted'],
    x = paid[0],
    textposition = "inside",
    textinfo = "value+percent initial"))

fig.add_trace(go.Funnel(
    name = 'Partner',
    orientation = "h",
    y = ['prequal_completed','prequal_rates_received','full_form_in_progress','full_form_completed','offer_accepted'],
    x = partner[0],
    textposition = "inside",
    textinfo = "value+percent initial"))

with open('p_graph.html', 'a') as f:
    f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
#plotly.offline.plot(fig, full_html=False,filename='name.html')
fig.show()

In [227]:
'''

Hover over the chart for more detail on percent from previous step and percent of total.

In analyzing this funnel we do not see considerable difference in the three marketing channels. The conversion rate
between steps is fairly consistent across all channells; the only rate that stands out as a possible outlier is the first
conversion rate for the partner channel from prequal completed to prequal rates received.

'''

'\n\nHover over the chart for more detail on percent from previous step and percent of total.\n\nIn analyzing this funnel we do not see considerable difference in the three marketing channels. The conversion rate\nbetween steps is fairly consistent across all channells; the only rate that stands out as a possible outlier is the first\nconversion rate for the partner channel from prequal completed to prequal rates received.\n\n'

In [228]:
'''

Group the funnel data by age. 

'''

age_group_df = df.groupby('age_group')['prequal_completed','prequal_rates_received','cosigner_invited','cosigner_active','full_form_in_progress','full_form_completed','offer_accepted'].sum()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [238]:
'''

Grouped dataframe

'''


age_group_df.head()

,prequal_completed,prequal_rates_received,full_form_in_progress,full_form_completed,offer_accepted
age_group,,,,,
18 to 25,426,237,123,78,17
25 to 35,3096,2118,806,569,249
35 to 55,1269,828,290,199,84
over 55,209,124,55,37,13


In [232]:
'''

Removed co-signers for ease of viewing. Will review co-signer effects later in analysis

'''

age_group_df = age_group_df.drop(['cosigner_invited','cosigner_active'], axis=1)

In [233]:
'''

Setup variables for plot

'''

age_18_to_25 = age_group_df[:1].values
age_25_to_35 = age_group_df[1:2].values
age_35_to_55 = age_group_df[2:3].values
age_55_over = age_group_df[3:4].values

In [274]:
'''

Plot Setup: Age

'''


from plotly import graph_objects as go

fig = go.Figure()

fig.add_trace(go.Funnel(
    name = '18 to 25',
    y = ['prequal_completed','prequal_rates_received','full_form_in_progress','full_form_completed','offer_accepted'],
    x = age_18_to_25[0],
    textinfo = "value+percent initial"))

fig.add_trace(go.Funnel(
    name = '25 to 35',
    orientation = "h",
    y = ['prequal_completed','prequal_rates_received','full_form_in_progress','full_form_completed','offer_accepted'],
    x = age_25_to_35[0],
    textposition = "inside",
    textinfo = "value+percent initial"))

fig.add_trace(go.Funnel(
    name = '35 to 55',
    orientation = "h",
    y = ['prequal_completed','prequal_rates_received','full_form_in_progress','full_form_completed','offer_accepted'],
    x = age_35_to_55[0],
    textposition = "inside",
    textinfo = "value+percent initial"))


fig.add_trace(go.Funnel(
    name = 'over 55',
    orientation = "h",
    y = ['prequal_completed','prequal_rates_received','full_form_in_progress','full_form_completed','offer_accepted'],
    x = age_55_over[0],
    textposition = "inside",
    textinfo = "value+percent initial"))


with open('p_graph.html', 'a') as f:
    f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
fig.show()

In [236]:
'''

Hover over the chart for more detail on percent from previous step and percent of total.

The age funnel shows disparate behavior for the four age groups. The oldest cohort over 55 has little to no users
in the funnel which would be expected for a student loan refinancing product. With that said, the second older cohort
between 35 and 55 comprises the second largest group and has the second highest offer conversion rate. The bulk of our
target market appears to be the young professional group between 25 - 35. This group has the highest rate of conversion.
The youngest cohort has a noticeable dropoff in the first step between prequal completed and prequal rates received. I'd
posit that this a result of the groups current stage of life - the group is likely still in college and not currently at
a point where refinancing is an attractive product for them.

Experiment 1:

An experiment that I would want to run is to create a targeted blog post and marketing campaign for young potential borrowers.
Ideally, I would want to create an informative post that highlights the importance of early loan refinancing as well as
paid traffic to the landing page. I expect that an effective post and campaign would improve conversion for the youth
cohort from 18 to 25 by stressing the importance of refinancing, highlighting the impact of co-signers and educating
a cohort that may be lacking in awareness. To evaluate the efficacy of the campaign, I would analyze the conversion data for
users that have been primed with the blog post and users that have not been primed with the blog post; of course, trying
to keep other conflating factors age, demographic, traffic source constant.


'''

"\n\nHover over the chart for more detail on percent from previous step and percent of total.\n\nThe age funnel shows disparate behavior for the four age groups. The oldest cohort over 55 has little to no users\nin the funnel which would be expected for a student loan refinancing product. With that said, the second older cohort\nbetween 35 and 55 comprises the second largest group and has the second highest offer conversion rate. The bulk of our\ntarget market appears to be the young professional group between 25 - 35. This group has the highest rate of conversion.\nThe youngest cohort has a noticeable dropoff in the first step between prequal completed and prequal rates received. I'd\nposit that this a result of the groups current stage of life - the group is likely still in college and not currently at\na point where refinancing is an attractive product for them.\n\nExperiment 1:\n\nAn experiment that I would want to run is to create a targeted blog post and marketing campaign for yo

In [242]:
'''

Group the funnel data by credit score. 

'''

fico_group_df = df.groupby('fico_group')['prequal_completed','prequal_rates_received','cosigner_invited','cosigner_active','full_form_in_progress','full_form_completed','offer_accepted'].sum()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [243]:
'''

Removed co-signers for ease of viewing. Will review co-signer effects later in analysis

'''

fico_group_df = fico_group_df.drop(['cosigner_invited','cosigner_active'], axis=1)

In [244]:
'''

Grouped dataframe

'''

fico_group_df.head()

,prequal_completed,prequal_rates_received,full_form_in_progress,full_form_completed,offer_accepted
fico_group,,,,,
630 to 689,853,247,141,85,12
690 to 719,704,484,157,104,31
720 to 850,2874,2568,952,675,319
less than 630,569,8,24,19,1


In [246]:
'''

Setup variables for plot

'''

fico_720_to_850 = fico_group_df[2:3].values
fico_690_to_719 = fico_group_df[1:2].values
fico_630_to_689 = fico_group_df[0:1].values
fico_less_than_630 = fico_group_df[3:4].values

In [275]:
'''

Plot Setup: Fico

'''

from plotly import graph_objects as go

fig = go.Figure()

fig.add_trace(go.Funnel(
    name = '720 to 850',
    y = ['prequal_completed','prequal_rates_received','full_form_in_progress','full_form_completed','offer_accepted'],
    x = fico_720_to_850[0],
    textinfo = "value+percent initial"))

fig.add_trace(go.Funnel(
    name = '690 to 719',
    orientation = "h",
    y = ['prequal_completed','prequal_rates_received','full_form_in_progress','full_form_completed','offer_accepted'],
    x = fico_690_to_719[0],
    textposition = "inside",
    textinfo = "value+percent initial"))

fig.add_trace(go.Funnel(
    name = '630 to 689',
    orientation = "h",
    y = ['prequal_completed','prequal_rates_received','full_form_in_progress','full_form_completed','offer_accepted'],
    x = fico_630_to_689[0],
    textposition = "inside",
    textinfo = "value+percent initial"))


fig.add_trace(go.Funnel(
    name = 'less than 630',
    orientation = "h",
    y = ['prequal_completed','prequal_rates_received','full_form_in_progress','full_form_completed','offer_accepted'],
    x = fico_less_than_630[0],
    textposition = "inside",
    textinfo = "value+percent initial"))


with open('p_graph.html', 'a') as f:
    f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
fig.show()

In [248]:
'''

Hover over the chart for more detail on percent from previous step and percent of total.

The high fico cohort is the most performant and comprises the majority of prequals completed. The funnel dips precipitously
as the fico scores drop; this would be as expected. That said we would hope that lower qualified individuals would be
able to enlist the help of a co-signer to enable them to view the loan products.

I am going to further analyze the main high value cohort in the below funnel.


'''

'\n\nHover over the chart for more detail on percent from previous step and percent of total.\n\nThe high fico cohort is the most performant and comprises the majority of prequals completed. The funnel dips precipitously\nas the fico scores drop; this would be as expected. That said we would hope that lower qualified individuals would be\nable to enlist the help of a co-signer to enable them to view the loan products.\n\nI am going to further analyze the main high value cohort in the below funnel.\n\n\n'

In [249]:
'''

Filter dataframe for only users with Fico scores above 720. Further, create a grouped dataframe by loan amount. I am
interested in seeing the conversion rate by loan amount when holding fico as a constant.

'''


loan_amount_group_df = df[df['fico_group'] == '720 to 850']
loan_amount_group_df = loan_amount_group_df.groupby('loan_amount_group')['prequal_completed','prequal_rates_received','cosigner_invited','cosigner_active','full_form_in_progress','full_form_completed','offer_accepted'].sum()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [250]:
'''

Removed co-signers for ease of viewing. Will review co-signer effects later in analysis

'''

loan_amount_group_df = loan_amount_group_df.drop(['cosigner_invited','cosigner_active'], axis=1)

In [251]:
'''

Grouped Dataframe

'''

loan_amount_group_df.head()

,prequal_completed,prequal_rates_received,full_form_in_progress,full_form_completed,offer_accepted
loan_amount_group,,,,,
25000 to 45000,672,623,229,167,80
45000 to 80000,687,619,230,160,73
less than 25000,717,603,235,165,84
over 80000,798,723,258,183,82


In [252]:
'''

Setup variables for plot

'''


loan_amount_over_80k = df2[3:4].values
loan_amount_45k_to_80k = df2[1:2].values
loan_amount_25k_to_45k = df2[0:1].values
loan_amount_less_than_25k = df2[2:3].values

In [276]:
'''

Plot Setup: High Fico and Loan Amount

'''


from plotly import graph_objects as go

fig = go.Figure()

fig.add_trace(go.Funnel(
    name = '80,000 or over',
    y = ['prequal_completed','prequal_rates_received','full_form_in_progress','full_form_completed','offer_accepted'],
    x = loan_amount_over_80k[0],
    textinfo = "value+percent initial"))

fig.add_trace(go.Funnel(
    name = '45,000 to 80,000',
    orientation = "h",
    y = ['prequal_completed','prequal_rates_received','full_form_in_progress','full_form_completed','offer_accepted'],
    x = loan_amount_45k_to_80k[0],
    textposition = "inside",
    textinfo = "value+percent initial"))

fig.add_trace(go.Funnel(
    name = '25,000 to 45,000',
    orientation = "h",
    y = ['prequal_completed','prequal_rates_received','full_form_in_progress','full_form_completed','offer_accepted'],
    x = loan_amount_25k_to_45k[0],
    textposition = "inside",
    textinfo = "value+percent initial"))


fig.add_trace(go.Funnel(
    name = 'less than 25,000',
    orientation = "h",
    y = ['prequal_completed','prequal_rates_received','full_form_in_progress','full_form_completed','offer_accepted'],
    x = loan_amount_less_than_25k[0],
    textposition = "inside",
    textinfo = "value+percent initial"))


with open('p_graph.html', 'a') as f:
    f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
fig.show()

In [254]:
'''

Hover over the chart for more detail on percent from previous step and percent of total.

Loan amount does not appear to have a noticable effect in conversion rate; high loan amounts and low loan amounts
appear to convert at the same rate. Further, the loan amounts appear to be normally distributed.


'''

'\n\nHover over the chart for more detail on percent from previous step and percent of total.\n\nLoan amount does not appear to have a noticable effect in conversion rate; high loan amounts and low loan amounts\nappear to convert at the same rate. Further, the loan amounts appear to be normally distributed.\n\n\n'

In [255]:
'''

Create dataframes for co-signed users and non-co-cosigned users


'''




cosigned_df = df[df['cosigner_invited'] == 1]
not_cosigned_df = df[df['cosigner_invited'] == 0]

cosigned_df.head()

,submission_id,marketing_channel,loan_amount,age,income,fico,prequal_completed,prequal_rates_received,cosigner_invited,cosigner_active,full_form_in_progress,full_form_completed,offer_accepted,age_group,fico_group,loan_amount_group
2,2348051,Partner,36172.0,32,90000.0,697,1,1,1,0,1,0,0,25 to 35,690 to 719,25000 to 45000
14,2166630,Paid,5865.0,26,47000.0,621,1,0,1,1,1,1,0,25 to 35,less than 630,less than 25000
21,2491533,Partner,53503.0,34,150000.0,805,1,1,1,0,1,1,0,25 to 35,720 to 850,45000 to 80000
22,2902448,Partner,73000.0,39,117000.0,850,1,1,1,0,0,0,0,35 to 55,720 to 850,45000 to 80000
23,2924874,Paid,128864.0,29,1.0,762,1,1,1,1,1,1,0,25 to 35,720 to 850,over 80000


In [256]:
'''

Create a group by object and sum events. I removed a few events in the funnel for ease of viewing

'''


cosigned_df = cosigned_df.groupby('fico_group')[['prequal_completed','cosigner_invited','cosigner_active','offer_accepted']].sum()
not_cosigned_df = not_cosigned_df.groupby('fico_group')[['prequal_completed','cosigner_invited','cosigner_active','offer_accepted']].sum()

In [257]:
'''

Grouped dataframe

'''


not_cosigned_df.head()

,prequal_completed,cosigner_invited,cosigner_active,offer_accepted
fico_group,,,,
630 to 689,657,0,0,6
690 to 719,560,0,0,18
720 to 850,2375,0,0,247
less than 630,476,0,0,0


In [258]:
'''

Create column of conversion rates from prequal completed to offer conversion

'''

not_cosigned_df['offer_conversion'] = not_cosigned_df['offer_accepted']/not_cosigned_df['prequal_completed']
cosigned_df['offer_conversion'] = cosigned_df['offer_accepted']/cosigned_df['prequal_completed']

In [259]:
'''

Grouped dataframe

'''


not_cosigned_df.head()

,prequal_completed,cosigner_invited,cosigner_active,offer_accepted,offer_conversion
fico_group,,,,,
630 to 689,657,0,0,6,0.009132
690 to 719,560,0,0,18,0.032143
720 to 850,2375,0,0,247,0.104000
less than 630,476,0,0,0,0.000000


In [261]:
'''

Setup variables for plot

'''


cosigned_df_rates = cosigned_df.offer_conversion.values
not_cosigned_df_rates = not_cosigned_df.offer_conversion.values


In [277]:
'''

Plot Setup: Co-Signed vs Not-Co-Signed

'''


fico_group =['Fico: 630 to 689', 'Fico: 690 to 719', 'Fico: 720 to 850','Fico: less than 630']

fig = go.Figure(data=[
    go.Bar(name='Co-Signed', x=fico_group, y=cosigned_df_rates),
    go.Bar(name='Not-Co-signed', x=fico_group, y=not_cosigned_df_rates)
])
# Change the bar mode
fig.update_layout(barmode='group')

with open('p_graph.html', 'a') as f:
    f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
fig.show()

In [263]:
'''

The bar chart highlights that offer conversion across all credit groups is higher for co-signed cohorts.


Experiment 2:

Create a marketing list and Facebook, Google Adwords audiences from the potential co-signers. Create targeted brand campaigns
for Credible to prime potential co-signers and reinforce the validity / safety of the platform.

The test group would have had their potential co-signers targeted and advertised to on digital ad platforms.
The control group would have no-targeted advertising to their potential co-signers.

The key metric to evaluate would be the conversion rate of invited co-signers to co-signers active.


'''

'\n\nThe bar chart highlights that offer conversion across all credit groups is higher for co-signed cohorts.\n\n\nExperiment 2:\n\nCreate a marketing list and Facebook, Google Adwords audiences from the potential co-signers. Create targeted brand campaigns\nfor Credible to prime potential co-signers and reinforce the validity / safety of the platform.\n\nThe test group would have had their potential co-signers targeted and advertised to on digital ad platforms.\nThe control group would have no-targeted advertising to their potential co-signers.\n\nThe key metric to evaluate would be the conversion rate of invited co-signers to co-signers active.\n\n\n'